# New Expiry Data Pull for rerunning of MVP scripts

Client provided new BQ query for expiry data. Query has been QAed 

In order to pull data by executing a query in a given .sql file, we need to first pull the data into a table within our locally owned GCP project (otherwise, we'll run into constraints).

This can be done via the following command line statement:

`bq query --use_legacy_sql=false --destination_table="radix2020:expiry.expiry_20180101_20211231_20210215" --flagfile="expiry_prepped_data.sql"`

This will create a new a new table expiry.expiry_20180101_20211231_20210215

Naming convention for expiry data pulls: expiry_mindate_maxdate_pulldate

In [15]:
library(bigrquery)

In [1]:
projname_str <- 'radix2020'
dbname_str <- 'expiry'
query_file <- 'expiry_prepped_data.sql'

In [2]:
today <- Sys.Date()

tblname_str_1 <- paste0('expiry_20180101_20211231_',format(today, format="%Y%m%d"))
tblloc_str_1 <- paste0(projname_str,':',dbname_str,'.',tblname_str_1)

command_str <-  paste0("bq query --use_legacy_sql=false --destination_table='",
                       tblloc_str_1,"' --flagfile='",query_file,"' ")


cat("Executing command:\n\t", command_str,"\n")

Executing command:
	 bq query --use_legacy_sql=false --destination_table='radix2020:expiry.expiry_20180101_20211231_20210222' --flagfile='expiry_prepped_data.sql'  


In [3]:
system(command_str)

cat("Created BQ table", tblloc_str_1 ,"\n")

Created BQ table radix2020:expiry.expiry_20180101_20211231_20210222 


In [4]:
command_str <- paste0("bq update --expiration 3600 '", tblloc_str_1,"'")
cat("Executing command:\n\t", command_str,"\n")

Executing command:
	 bq update --expiration 3600 'radix2020:expiry.expiry_20180101_20211231_20210222' 


In [5]:
system(command_str)
cat("Updated BQ table", tblloc_str_1 ," to be automatically deleted in 1 hour (",format(Sys.time()+ 1*60*60, tz="America/Los_Angeles",usetz=TRUE),")\n")

Updated BQ table radix2020:expiry.expiry_20180101_20211231_20210222  to be automatically deleted in 1 hour ( 2021-02-22 15:16:38 PST )


# Test/Train split

Client has requested a 10%/45%/45% (Test/Train/Train) split 

with a 5 quarter time horizon... leading to an est. 560K/2.5M/2.5M obs

09/28/2019 - 12/27/2020 =5Q

https://www.oreilly.com/content/repeatable-sampling-of-data-sets-in-bigquery-for-machine-learning/
https://stackoverflow.com/questions/46019624/how-to-do-repeatable-sampling-in-bigquery-standard-sql
https://stackoverflow.com/questions/46019624/how-to-do-repeatable-sampling-in-bigquery-standard-sql

In [9]:
maxdate <- today - 50
mindate <- maxdate - 456

In [10]:
# Create local dir for data
directory <- paste0('../../data/output/datapull_', format(today, format="%Y%m%d"))
dir.create(directory, showWarnings = FALSE)

In [11]:
tblname_str_2 <- paste0('expiry_',format(mindate, format="%Y%m%d"),'_',format(maxdate, format="%Y%m%d") ,'_test')
tblloc_str_2 <- paste0(projname_str,':',dbname_str,'.',tblname_str_2)

query_str <- gsub("[\r\n]", " ", paste0("SELECT * FROM  ",dbname_str,'.',tblname_str_1," t WHERE
  DATE(expiry_date) BETWEEN \"",mindate,"\" AND \"",maxdate,"\" AND renewed_count=1 AND  
  ABS(HASH(expiry_date)) % 100 < 10"))

command_str <- paste0("bq query --destination_table='", tblloc_str_2,"' '", query_str ,"'")
cat("Executing command:\n\t", command_str,"\n")

Executing command:
	 bq query --destination_table='radix2020:expiry.expiry_20191005_20210103_test' 'SELECT * FROM  expiry.expiry_20180101_20211231_20210222 t WHERE   DATE(expiry_date) BETWEEN "2019-10-05" AND "2021-01-03" AND renewed_count=1 AND     ABS(HASH(expiry_date)) % 100 < 10' 


In [12]:
system(command_str)

cat("Created BQ table", tblloc_str_2 ,"\n")

Created BQ table radix2020:expiry.expiry_20191005_20210103_test 


In [17]:
# Write data to local
# https://community.exploratory.io/t/google-bigquery-import-fails-with-invalid-value-at-start-index-type-uint64-1e-05-invalid/1901
options(scipen = 20)

eval(call("<-", as.name(tblname_str_2), 
          bq_table_download(bq_project_query(projname_str, 
                                             paste0("SELECT * FROM  ",dbname_str,'.',tblname_str_2)
                                            ))))
saveRDS(get(tblname_str_2),file = paste0(directory,'/',tblname_str_2,'.RDS'), 
        compress=TRUE)


In [27]:
tblname_str_2 <- paste0('expiry_',format(mindate, format="%Y%m%d"),'_',format(maxdate, format="%Y%m%d") ,'_train1')
tblloc_str_2 <- paste0(projname_str,':',dbname_str,'.',tblname_str_2)

query_str <- gsub("[\r\n]", " ", paste0("SELECT * FROM  ",dbname_str,'.',tblname_str_1," t WHERE
  DATE(expiry_date) BETWEEN \"",mindate,"\" AND \"",maxdate,"\" AND renewed_count=1 AND  
  ABS(HASH(expiry_date)) % 100 >= 10 AND  
  ABS(HASH(expiry_date)) % 100 < 55"))

command_str <- paste0("bq query --destination_table='", tblloc_str_2,"' '", query_str ,"'")
cat("Executing command:\n\t", command_str,"\n")

Executing command:
	 bq query --destination_table='radix2020:expiry.expiry_20191005_20210103_train1' 'SELECT * FROM  expiry.expiry_20180101_20211231_20210222 t WHERE   DATE(expiry_date) BETWEEN "2019-10-05" AND "2021-01-03" AND renewed_count=1 AND     ABS(HASH(expiry_date)) % 100 >= 10 AND     ABS(HASH(expiry_date)) % 100 < 55' 


In [28]:
system(command_str)

cat("Created BQ table", tblloc_str_2 ,"\n")

Created BQ table radix2020:expiry.expiry_20191005_20210103_train1 


In [29]:
# Write data to local
# https://community.exploratory.io/t/google-bigquery-import-fails-with-invalid-value-at-start-index-type-uint64-1e-05-invalid/1901
options(scipen = 20)

eval(call("<-", as.name(tblname_str_2), 
          bq_table_download(bq_project_query(projname_str, 
                                             paste0("SELECT * FROM  ",dbname_str,'.',tblname_str_2)
                                            ))))
saveRDS(get(tblname_str_2),file = paste0(directory,'/',tblname_str_2,'.RDS'), 
        compress=TRUE)

In [32]:
tblname_str_2 <- paste0('expiry_',format(mindate, format="%Y%m%d"),'_',format(maxdate, format="%Y%m%d") ,'_train2')
tblloc_str_2 <- paste0(projname_str,':',dbname_str,'.',tblname_str_2)

query_str <- gsub("[\r\n]", " ", paste0("SELECT* FROM ",dbname_str,'.',tblname_str_1," t WHERE
  DATE(expiry_date) BETWEEN \"",mindate,"\" AND \"",maxdate,"\" AND renewed_count=1 AND  
  ABS(HASH(expiry_date)) % 100 >= 55 "))

command_str <- paste0("bq query --destination_table='", tblloc_str_2,"' '", query_str ,"'")
cat("Executing command:\n\t", command_str,"\n")

Executing command:
	 bq query --destination_table='radix2020:expiry.expiry_20191005_20210103_train2' 'SELECT* FROM expiry.expiry_20180101_20211231_20210222 t WHERE   DATE(expiry_date) BETWEEN "2019-10-05" AND "2021-01-03" AND renewed_count=1 AND     ABS(HASH(expiry_date)) % 100 >= 55 ' 


In [33]:
system(command_str)

cat("Created BQ table", tblloc_str_2 ,"\n")

Created BQ table radix2020:expiry.expiry_20191005_20210103_train2 


In [34]:
# Write data to local
# https://community.exploratory.io/t/google-bigquery-import-fails-with-invalid-value-at-start-index-type-uint64-1e-05-invalid/1901
options(scipen = 20)

eval(call("<-", as.name(tblname_str_2), 
          bq_table_download(bq_project_query(projname_str, 
                                             paste0("SELECT * FROM  ",dbname_str,'.',tblname_str_2)
                                            ))))
saveRDS(get(tblname_str_2),file = paste0(directory,'/',tblname_str_2,'.RDS'), 
        compress=TRUE)

In [36]:
system(paste0("gsutil cp -r ",directory," gs://data_outputt/output/"))

In [6]:
sql <- paste("SELECT * FROM `radix2020.expiry.expiry_20200902_20201102_20201127`")

expiry_20200902_20201102_20201127 <- bq_table_download(bq_project_query("radix2020", sql))

In [7]:
dim(expiry_20200902_20201102_20201127)

[1] 943779     30

In [8]:
saveRDS(expiry_20200902_20201102_20201127,"../../data/output/datapull_20201127/expiry_20200902_20201102_20201127")


In [ ]:
# Create cloud storage dir for data 
bucket <- 'gs://data_outputt/output/'
system(paste0("gsutil cp -r ",directory, " ", bucket))

In [13]:
library(bigrquery)

In [30]:
query <- paste("
SELECT
*
FROM
  `radix2020.expiry.expiry_20180101_20211231_20210215` t
WHERE
  DATE(expiry_date) BETWEEN '2020-12-26' AND '2020-12-27' AND renewed_count=1 AND  
  MOD(ABS(FARM_FINGERPRINT(FORMAT('%t', t))), 10) < 1
")

query
test_df <- bq_table_download(bq_project_query("radix2020", query))

# sql <- paste("SELECT * FROM `radix2020.expiry.new_test`")


[1] "\nSELECT\n*\nFROM\n  `radix2020.expiry.expiry_20180101_20211231_20210215` t\nWHERE\n  DATE(expiry_date) BETWEEN '2020-12-26' AND '2020-12-27' AND renewed_count=1 AND  \n  MOD(ABS(FARM_FINGERPRINT(FORMAT('%t', t))), 10) < 1\n"

In [31]:
dim(test_df)
head(test_df)

[1] 2335   30

domain_id,renewed_count,renewal_type,expiry_date,creation_date,creation_time,tld,domain,registrar,reseller,⋯,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,predicted_renewal_rate
<int>,<int>,<chr>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<date>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<dbl>
156057156,1,FirstTime,2020-12-26,2019-12-26,2019-12-26 08:57:44,press,todayis.press,Porkbun,Porkbun,⋯,2020-12-26,49,3.77,todayis,1,1,7,l,7l,0.048
156188916,1,FirstTime,2020-12-27,2019-12-27,2019-12-27 01:41:59,space,roama.space,Google,Google,⋯,NA,NA,4.08,roama,1,5,5,l,5l,0.698
156159101,1,FirstTime,2020-12-26,2019-12-26,2019-12-26 21:35:07,website,saudenatural.website,Hostinger,Hostinger,⋯,2020-12-21,15,5.28,saudenatural,1,11,12,l,12l,0.079
156162086,1,FirstTime,2020-12-26,2019-12-26,2019-12-26 21:57:29,fun,forhumanitywithlovellc.fun,Network Solutions,Network Solutions,⋯,2020-12-26,15,4.18,forhumanitywithlovellc,1,1,22,l,22l,0.482
156544421,1,FirstTime,2020-12-27,2019-12-27,2019-12-27 15:31:23,pw,financeshome.pw,Key-Systems,Key-Systems,⋯,2020-12-27,15,9.62,financeshome,1,10,12,l,12l,0.239
156604436,1,FirstTime,2020-12-27,2019-12-27,2019-12-27 21:56:50,space,onemotion.space,Porkbun,Porkbun,⋯,2020-04-10,15,8.04,onemotion,1,1,9,l,9l,0.276


In [34]:
bq_table_download(bq_project_query("radix2020", "select count(*) from  `radix2020.expiry.expiry_20180101_20211231_20210215` WHERE
  DATE(expiry_date) BETWEEN '2020-12-26' AND '2020-12-27' AND renewed_count=1 ", use_legacy_sql = FALSE))


f0_
<int>
22886


In [35]:
22886*0.1

[1] 2288.6